In [2]:
from model import TMapper, PhiMapper, OmegaMapper, UnbalancedLoss
import torch
from collections import Counter

In [3]:
from model import Hellinger, Jensen_Shannon, KL_dual, Pearson_xi

In [4]:
import torch.nn.functional as F
import torchvision
from torchvision.datasets import MNIST
from torch.utils.data import Dataset, DataLoader

In [5]:
def cost_matrix(x:torch.Tensor, y:torch.Tensor) -> torch.Tensor:
    return torch.norm(x[:, None] - y, dim=2, p=2)

def mass_variation(s):
    return (s - 1)**2

In [6]:
T = TMapper(784, 784).float().cuda()
Xi = PhiMapper(784, 1).float().cuda()
f = OmegaMapper(784, 1).float().cuda()
loss = UnbalancedLoss(10, 5, cost_matrix, mass_variation, KL_dual)
w_optim = torch.optim.SGD(f.parameters(), lr=1e-2)
t_optim = torch.optim.SGD(T.parameters(), lr=1e-2)
xi_optim = torch.optim.SGD(Xi.parameters(), lr=1e-2)

In [7]:
sampler1 = torch.utils.data.SubsetRandomSampler(torch.arange(30000))
sampler2 = torch.utils.data.SubsetRandomSampler(torch.arange(30000, 60000))

In [8]:
def target_transform(x):
    return x.type(torch.double).view(1)

In [9]:
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        return torch.reshape(img, self.new_size)

In [10]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    ReshapeTransform((-1, ))
])

dataset = MNIST(root='.', download=True, train=True, transform=transform, target_transform=target_transform)

dl_1 = DataLoader(dataset, batch_size=1000, sampler=sampler1)
dl_2 = DataLoader(dataset, batch_size=1000, sampler=sampler2)

In [15]:
def train_loop(data_loader_1: torch.utils.data.DataLoader, data_loader_2: torch.utils.data.DataLoader):
    for (X, Z_1), (Y, Z) in zip(data_loader_1, data_loader_2):
        
        X = X.float().cuda()
        Y = Y.float().cuda()
        Z = Z.float().cuda()
        
        w_optim.zero_grad()
        t_optim.zero_grad()
        xi_optim.zero_grad()
        print(X.shape)
        print(T(X, Z).shape)
        loss_value = -loss.compute(X, Z, Y, T(X, Z), Xi(X), f(Y), f(T(X, Z)))
        print('-loss value {}'.format(loss_value))
        loss_value.backward()
        w_optim.step()
        
        t_optim.zero_grad()
        xi_optim.zero_grad()
        loss_value = loss.compute(X, Z, Y, T(X, Z), Xi(X), f(Y), f(T(X, Z)))
        print('loss value {}'.format(loss_value))
        loss_value.backward()
        t_optim.step()
        xi_optim.step()

In [16]:
train_loop(dl_1, dl_2)

torch.Size([1000, 784])
torch.Size([1000, 784])
-loss value -8.61545181274414


RuntimeError: CUDA out of memory. Tried to allocate 2.92 GiB (GPU 0; 7.92 GiB total capacity; 5.92 GiB already allocated; 1.40 GiB free; 970.50 KiB cached)